In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
import time
import pickle
import napari
import numpy as np
import pandas as pd

from skimage.io import imread

import btrack
from btrack.constants import BayesianUpdates

sys.path.append('../libraries')
import input_functions as inp_f

In [2]:
info_file_path = r'Z:\Sonja\210910_endo_meso_52\210910_endo_meso_52_info.txt'

In [14]:
# read the file
info_file = open(info_file_path, 'r')
info_lines = info_file.readlines()
info_file.close()

# read info about the data frame
exp_dir,df_name = inp_f.read_df_info(info_lines)

df_dir = os.path.join(exp_dir,'df')
save_dir = df_dir

frames_to_exclude = inp_f.read_frames_2_exclude(info_lines)
frames_to_exclude = eval(frames_to_exclude)

modelPath = os.path.join(exp_dir,'code','libraries','cell_config.json')

## Read in the data frame objects data

In [15]:
data_df = pd.read_pickle(os.path.join(df_dir,df_name))
len(data_df)

223543

In [17]:
'''
data_df.drop(inplace=True,axis=1,columns=['track_id', 'x', 'y', 'parent', 'generation', 'root', 'accepted',
       'rejected', 'promise'])
'''

## Tracking proper

In [18]:
n_part = 4
part_len = int(np.max(data_df.t)/n_part)

In [19]:
track_list = []
for n in range(n_part):
    
    #calculate boundaries
    start = n * part_len
    stop = (n+1) * part_len
    if n == (n_part - 1):
        stop = len(data_df)
        
    # calculate offset
    if n==0:
        offset = 0
    else:
        offset = np.max(data[:,0])
    
    # choose objects 
    sel_vector = [(not(x in frames_to_exclude) and (x>start) and (x<stop)) for x in data_df.t]

    objects_gen = data_df.loc[sel_vector,['label','area','centroid-1','centroid-0','major_axis_length','minor_axis_length','t']]

    objects_gen.columns=['ID', 'area', 'x', 'y', 'major_axis_length','minor_axis_length','t']
    objects_gen['z']=0
    objects_gen['label']=5
    objects_gen['prob']=0
    objects_gen['dummy']=False
    objects_gen['states']=0

    # initialise a tracker session using a context manager
    with btrack.BayesianTracker() as tracker:

        # configure the tracker using a config file
        tracker.configure_from_file(modelPath)

        # approximate
        tracker.update_method = BayesianUpdates.APPROXIMATE
        tracker.max_search_radius = 200

        # append the objects to be tracked
        tracker.append(objects_gen)

        # set the volume (Z axis volume is set very large for 2D data)
        tracker.volume=((0, data_df.size_x[0]), (0, data_df.size_y[0]), (-1e5, 1e5))

        # track them (in interactive mode)
        tracker.track_interactive(step_size=100)
        
        # generate hypotheses and run the global optimizer
        tracker.optimize()

        # optional: get the data in a format for napari
        data, properties, graph = tracker.to_napari(ndim=2)
        
    # add offset
    data[:,0] = data[:,0] + offset
    #data[:,1] = data[:,1]
        
    # collect data
    temp = pd.DataFrame(data,columns=['track_id','t','x','y'])
    temp['parent'] = properties['parent'] + offset
    temp['generation'] = properties['generation']
    temp['root'] = properties['root'] + offset
    
    track_list.append(temp)
    
trackDataAll = pd.concat(track_list,ignore_index = True)

[INFO][2021/12/08 12:48:18 PM] Loaded btrack: C:\Users\kmkedz\AppData\Roaming\Python\Python37\site-packages\btrack\libs\libtracker.DLL
[INFO][2021/12/08 12:48:18 PM] btrack (v0.4.2) library imported
[INFO][2021/12/08 12:48:18 PM] Setting max XYZ search radius to: 100
[INFO][2021/12/08 12:48:18 PM] Starting BayesianTracker session
[INFO][2021/12/08 12:48:18 PM] Loading configuration file: Z:\Sonja\210910_endo_meso_52\code\libraries\cell_config.json
[INFO][2021/12/08 12:48:18 PM] Loading motion model: b'cell_motion'
[INFO][2021/12/08 12:48:18 PM] Setting Bayesian update method to: BayesianUpdates.APPROXIMATE
[INFO][2021/12/08 12:48:18 PM] Setting max XYZ search radius to: 200
[INFO][2021/12/08 12:48:18 PM] Objects are of type: <class 'pandas.core.frame.DataFrame'>
[INFO][2021/12/08 12:48:19 PM] Set volume to ((0, 2048), (0, 2048), (-100000.0, 100000.0))
[INFO][2021/12/08 12:48:19 PM] Starting tracking... 
[INFO][2021/12/08 12:48:19 PM] Tracking objects in frames 0 to 99 (of 258)...
[INFO

[INFO][2021/12/08 12:53:18 PM]  - Found 8687 tracks in 774 frames (in 0.0s)
[INFO][2021/12/08 12:53:18 PM]  - Inserted 7629 dummy objects to fill tracking gaps
[INFO][2021/12/08 12:53:18 PM] Loading hypothesis model: cell_hypothesis
[INFO][2021/12/08 12:53:18 PM] Calculating hypotheses (relax: True)...
[INFO][2021/12/08 12:53:18 PM] Setting up constraints matrix for global optimisation...
[INFO][2021/12/08 12:53:20 PM] Optimizing...
[INFO][2021/12/08 12:53:25 PM] Optimization complete. (Solution: optimal)
[INFO][2021/12/08 12:53:25 PM]  - Fates.FALSE_POSITIVE: 4204 (of 8687)
[INFO][2021/12/08 12:53:25 PM]  - Fates.LINK: 2372 (of 4463)
[INFO][2021/12/08 12:53:25 PM]  - Fates.DIVIDE: 207 (of 752)
[INFO][2021/12/08 12:53:25 PM]  - Fates.INITIALIZE_BORDER: 119 (of 439)
[INFO][2021/12/08 12:53:25 PM]  - Fates.INITIALIZE_LAZY: 1578 (of 8248)
[INFO][2021/12/08 12:53:25 PM]  - Fates.TERMINATE_BORDER: 111 (of 438)
[INFO][2021/12/08 12:53:25 PM]  - Fates.TERMINATE_BACK: 14 (of 36)
[INFO][2021/12

## Merging objects and tracking information

In [20]:
len(trackDataAll)

263046

In [21]:
allData = pd.merge(left=data_df,right=trackDataAll,left_on=['centroid-0','centroid-1','t'],right_on=['x','y','t'],how='left')

In [22]:
print(f'Number of all objects: {len(allData)}')

Number of all objects: 223543


In [23]:
allData.head()

,label,area,centroid-0,centroid-1,orientation,major_axis_length,minor_axis_length,bbox-0,bbox-1,bbox-2,...,mean_intensity-0_ring,mean_intensity-1_ring,size_x,size_y,track_id,x,y,parent,generation,root
0,7,1338,37.109118,1727.743647,-0.369780,54.154746,32.189471,8,1708,62,...,224.069307,145.935644,2048,2048,NaN,NaN,NaN,NaN,NaN,NaN
1,8,1373,40.700655,604.683176,-1.136684,48.529689,36.937616,22,579,63,...,226.690815,141.375162,2048,2048,NaN,NaN,NaN,NaN,NaN,NaN
2,9,1246,42.348315,389.571429,0.927465,53.063363,30.931133,24,364,64,...,233.304795,141.203767,2048,2048,NaN,NaN,NaN,NaN,NaN,NaN
3,10,1119,47.858803,847.581769,1.024182,53.249772,27.858318,30,826,67,...,241.000000,147.360324,2048,2048,NaN,NaN,NaN,NaN,NaN,NaN
4,11,1532,49.441906,1182.359661,-0.666132,57.934427,34.102222,23,1159,74,...,234.783175,146.752370,2048,2048,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
allData.columns

Index(['label', 'area', 'centroid-0', 'centroid-1', 'orientation',
       'major_axis_length', 'minor_axis_length', 'bbox-0', 'bbox-1', 'bbox-2',
       'bbox-3', 'image', 'mean_intensity-0_nuc', 'mean_intensity-1_nuc',
       'file', 't', 'centroid-0_ring', 'centroid-1_ring',
       'mean_intensity-0_ring', 'mean_intensity-1_ring', 'size_x', 'size_y',
       'track_id', 'x', 'y', 'parent', 'generation', 'root'],
      dtype='object')

## Define promising tracks

In [25]:
allData['accepted'] = False
allData['rejected'] = False
allData['promise'] = False

# mark tracks longer than 100 as promising
tracks_set = set(allData.track_id)

for track in tracks_set:
    
    track_len = np.sum(allData.track_id==track)
    
    if (track_len > 50):
        
        allData.loc[allData.track_id==track,'promise'] = True

In [26]:
# save df
allData.to_pickle(os.path.join(df_dir,df_name))
allData.to_csv(os.path.join(df_dir,df_name.replace('pkl','csv')),index=False)